In [90]:
library(data.table)
#library(dplyr)
library(geosphere)
library(stringr)
library(SearchTrees)
library(randomForest)

In [70]:
# get blocks
get_blocks <- function(state_number, block_directory) {
    block_file_name <- paste("tl_2010_", state_number, "_tabblock10.txt", sep="")
    block_file_loc <- file.path(block_directory, block_file_name)

    # blockfile csvs are pipe delimited
    return(read.csv(block_file_name, sep="|", stringsAsFactors=FALSE))
}
# 12 is the census value for FL
state_blocks <- get_blocks(state_number='12', block_directory='./')
head(state_blocks)

STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE10,GEOID10,NAME10,MTFCC10,UR10,UACE10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,POP10,HU10,Longitude,Latitude
12,1,2205,4006,1.2001e+14,Block 4006,G5040,NA,NA,S,10458,0,29.59290,-82.43768,19,8,-82.43766,29.59292
12,1,2205,4021,1.2001e+14,Block 4021,G5040,NA,NA,S,8429,0,29.59521,-82.43767,21,8,-82.43768,29.59520
12,1,1604,4040,1.2001e+14,Block 4040,G5040,NA,NA,S,4595,0,29.63753,-82.39491,20,12,-82.39493,29.63753
12,1,1604,4029,1.2001e+14,Block 4029,G5040,NA,NA,S,6249,0,29.64370,-82.40072,19,10,-82.40069,29.64369
12,1,2204,2027,1.2001e+14,Block 2027,G5040,NA,NA,S,123118,0,29.67882,-82.49133,52,21,-82.49156,29.67901
12,1,2204,2048,1.2001e+14,Block 2048,G5040,NA,NA,S,11802,0,29.65910,-82.48407,34,14,-82.48427,29.65871


In [71]:
# reduce columns and get only those blocks where people seem to live
# "STATEFP10"  "COUNTYFP10" "TRACTCE10"  "BLOCKCE10"  "INTPTLAT10" "INTPTLON10" "HU10"
state_blocks <- state_blocks[c(1,2,3,4,13,14,16)]
state_blocks <- subset(state_blocks, state_blocks$HU10 > 0)

In [72]:
# pad dataframe columns and add 'geoid' to dataframe which is the concatination of the four key identifiers
state_blocks$STATEFP10 <- str_pad(state_blocks$STATEFP10, width=2, side='left', pad='0')
state_blocks$COUNTYFP10 <- str_pad(state_blocks$COUNTYFP10, width=3, side='left', pad='0')
state_blocks$TRACTCE10 <- str_pad(state_blocks$TRACTCE10, width=6, side='left', pad='0')
state_blocks$BLOCKCE10 <- str_pad(state_blocks$BLOCKCE10, width=4, side='left', pad='0')
state_blocks$geoid <- str_c(state_blocks$STATEFP10, 
                            state_blocks$COUNTYFP10, 
                            state_blocks$TRACTCE10, 
                            state_blocks$BLOCKCE10)
head(state_blocks)

STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE10,INTPTLAT10,INTPTLON10,HU10,geoid
12,001,002205,4006,29.59290,-82.43768,8,120010022054006
12,001,002205,4021,29.59521,-82.43767,8,120010022054021
12,001,001604,4040,29.63753,-82.39491,12,120010016044040
12,001,001604,4029,29.64370,-82.40072,10,120010016044029
12,001,002204,2027,29.67882,-82.49133,21,120010022042027
12,001,002204,2048,29.65910,-82.48407,14,120010022042048


In [ ]:
# Read in sinkhole incidents
# Data Dictionary http://geodata.dep.state.fl.us/datasets/194ac159bd1e4be294b14dd882d42dcf_0
#florida_incidents <- read.csv(file = 'Florida_Subsidence_Incident_Reports.csv', stringsAsFactors=FALSE)
#florida_incidents <- florida_incidents[c('OBJECTID', 
 #                                        'LATDD', 
  #                                       'LONGDD', 
   #                                      'EVENT_DATE', 
    #                                     'SIZDIM', 
     #                                    'SINSHAPE', 
      #                                   'SINLNGTH', 
       #                                  'SINWIDTH', 
        #                                 'SINDEPTH',
         #                                'SUBRATE', 
          #                               'PROPDAM', 
           #                              'REPAIR_S')]
#head(florida_incidents)

In [ ]:
#florida_lon_lat <- as.data.frame(florida_incidents[,c('LONGDD', 'LATDD')])
#incident_tree <- createTree(florida_lon_lat, maxDepth=50)

#blkPt <- as.matrix(state_blocks[,c(3,2)]) #

#t1 <- Sys.time()
#NN <- knnLookup(incident_tree, newdat=blkPt, k=2) # euclidean dist miss nearest geodesic pt
#t2 <- Sys.time()
#t2-t1

In [ ]:
#length(rectLookup(incident_tree, xlims=c(-82.43768 - 0.2, -82.43768 + 0.2), ylims=c(29.59290 - 0.2, 29.59290 + 0.2)))
#nrow(slice(florida_incidents, rectLookup(incident_tree, xlims=c(-82.43768 - 0.2, -82.43768 + 0.2), ylims=c(29.59290 - 0.2, 29.59290 + 0.2))))

#incidents_within_13_miles <- function(long_lat_pt) {
    # returns the incidents within ~13 miles of long_lat_pt.  All calculations in lat/long or lat/long approximations
#    xmin_max <- c(long_lat_pt[1] - 0.2, long_lat_pt[1] + 0.2)
 #   ymin_max <- c(long_lat_pt[2] - 0.2, long_lat_pt[2] + 0.2)
  #  thirteen_mile_rect <- rectLookup(incident_tree, xlims=xmin_max, ylims=ymin_max)
   # return(slice(florida_incidents, thirteen_mile_rect))
#}

#incidents_within_x_miles <- function(long_lat_pt, incidents, x=1) {
#    great_circle_dist <- function(x) { return(distGeo(long_lat_pt, x) * 0.000621371) } # in miles
#    incidents$dist <- apply(incidents[,c('LONGDD', 'LATDD')], 1, great_circle_dist)
#    less_than_x_mile <- filter(incidents, incidents$dist < x)
#    return(less_than_x_mile)
#}

#num_incidents_within_x_miles <- function(incidents) {return(nrow(incidents))}

#pt <- c(-82.43768, 29.59290)
#lt_1_mile <- function(pt) {return(num_incidents_within_x_miles(incidents_within_x_miles(pt, incidents_within_13_miles(pt))))}
#lt_875_mile <- function(pt) {return(num_incidents_within_x_miles(incidents_within_x_miles(pt, incidents_within_13_miles(pt), x=0.875)))}
#lt_75_mile <- function(pt) {return(num_incidents_within_x_miles(incidents_within_x_miles(pt, incidents_within_13_miles(pt), x=0.75)))}
#lt_625_mile <- function(pt) {return(num_incidents_within_x_miles(incidents_within_x_miles(pt, incidents_within_13_miles(pt), x=0.625)))}
#lt_5_mile <- function(pt) {return(num_incidents_within_x_miles(incidents_within_x_miles(pt, incidents_within_13_miles(pt), x=0.5)))}
#lt_375_mile <- function(pt) {return(num_incidents_within_x_miles(incidents_within_x_miles(pt, incidents_within_13_miles(pt), x=0.375)))}
#lt_25_mile <- function(pt) {return(num_incidents_within_x_miles(incidents_within_x_miles(pt, incidents_within_13_miles(pt), x=0.25)))}
#lt_125_mile <- function(pt) {return(num_incidents_within_x_miles(incidents_within_x_miles(pt, incidents_within_13_miles(pt), x=0.125)))}

In [ ]:
# Calculation of the various distances from the centroid by which to calculate the probability.
# If I were better with R this could probably be sped up tremendously ~10 hours with the below method 
# on my i7 laptop with 8GB of Ram overnight
#t1 <- Sys.time()
#lat_lng <- state_blocks[c("INTPTLON10", "INTPTLAT10")]
#state_blocks$num_sinkholes_within_mile <- apply(lat_lng, 1, lt_1_mile)
#state_blocks$num_sinkholes_within_875 <- apply(lat_lng, 1, lt_875_mile)
#state_blocks$num_sinkholes_within_75 <- apply(lat_lng, 1, lt_75_mile)
#state_blocks$num_sinkholes_within_625 <- apply(lat_lng, 1, lt_625_mile)
#state_blocks$num_sinkholes_within_5 <- apply(lat_lng, 1, lt_5_mile)
#state_blocks$num_sinkholes_within_375 <- apply(lat_lng, 1, lt_375_mile)
#state_blocks$num_sinkholes_within_25 <- apply(lat_lng, 1, lt_25_mile)
#state_blocks$num_sinkholes_within_125 <- apply(lat_lng, 1, lt_125_mile)
#t2 <- Sys.time()
#t2 - t1

In [9]:
#head(state_blocks, n=5)

In [10]:
# Write output for backup of time consuming step and for consumption later as backup
#write.table(state_blocks, file="state_blocks_num_sinkholes_output_bak.csv", sep=",")
#state_blocks <- read.table(file="state_blocks_num_sinkholes_output_bak.csv", sep=",")

In [11]:
sinkhole_probabilities <- read.table("Sinkhole_Probabilities.csv", sep=',', header=TRUE)
sinkhole_probabilities <- sinkhole_probabilities[1:11, -c(10, 11, 12, 13)] # Get rid of excess columns
sinkhole_probabilities <- sinkhole_probabilities[,c('X0.75', 'X0.5', 'X0.25', 'X0.125')]
sinkhole_probabilities

X0.75,X0.5,X0.25,X0.125
0.000016,0.000016,0.000016,0.000016
0.000120,0.000243,0.000760,0.002434
0.000144,0.000304,0.001023,0.003607
0.000160,0.000351,0.001254,0.004602
0.000177,0.000400,0.001435,0.005549
0.000192,0.000442,0.001594,0.006400
0.000209,0.000483,0.001794,0.007341
0.000221,0.000525,0.001935,0.008427
0.000235,0.000558,0.002079,0.009020
0.000249,0.000589,0.002220,0.009450


In [32]:
# read old centroid with calculations in
calculated_sinkholes <- read.csv("state_blocks_num_sinkholes_output_bak.csv")
head(calculated_sinkholes)

# have to join tracts to zips 
# Obtained data here: https://www.huduser.gov/portal/datasets/usps_crosswalk.html
tract_to_csv <- read.csv("TRACT_ZIP_062016.csv")
head(tract_to_csv)

sprintf("Tracts in TRACTS_TO_ZIP file: %s", length(unique(tract_to_csv$TRACT)))
sprintf("Tracts in Florida Calculated Sinkholes (HU>0): %s", length(unique(calculated_sinkholes$TRACTCE10)))

calculated_sinkholes$state_county_tract <- substr(calculated_sinkholes$geoid, 1, 11)
tracts_with_zips <- merge(x = calculated_sinkholes, y = tract_to_csv, 
                          by.x = "state_county_tract", by.y='TRACT', 
                          all.x = TRUE)
sprintf("Florida Tracts (HU>0) joined with zips: %s", nrow(tracts_with_zips))
sprintf("Unique Zips in join: %s", length(unique(tracts_with_zips$ZIP)))

sprintf("Calculated sinkholes nrows: %s", nrow(calculated_sinkholes))
sprintf("Merged Sinkholes File nrow: %s", nrow(tracts_with_zips))

print("Note: there are more zips than tracts which is why the count has expanded so rapidly.  We will deal with that below in the max risk function")

X,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE10,INTPTLAT10,INTPTLON10,HU10,geoid,num_sinkholes_within_mile,num_sinkholes_within_875,num_sinkholes_within_75,num_sinkholes_within_5,num_sinkholes_within_375,num_sinkholes_within_25,num_sinkholes_within_125,idx,state_county_tract
1,12,1,2205,4006,29.59290,-82.43768,8,1.2001e+14,2,2,2,2,0,0,0,1,1.2012e+10
2,12,1,2205,4021,29.59521,-82.43767,8,1.2001e+14,2,2,2,2,0,0,0,2,1.2012e+10
3,12,1,1604,4040,29.63753,-82.39491,12,1.2001e+14,1,1,0,0,0,0,0,3,1.2012e+10
4,12,1,1604,4029,29.64370,-82.40072,10,1.2001e+14,0,0,0,0,0,0,0,4,1.2012e+10
5,12,1,2204,2027,29.67882,-82.49133,21,1.2001e+14,1,1,1,1,0,0,0,5,1.2012e+10
6,12,1,2204,2048,29.65910,-82.48407,14,1.2001e+14,0,0,0,0,0,0,0,6,1.2012e+10


X,TRACT,ZIP,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
0,1001020100,36067,1.00000000,1.000000000,1.00000000,1.00000000
1,1001020200,36067,0.41019956,0.425174825,0.94736842,0.42054521
2,1001020200,36008,0.01197339,0.005594406,0.00000000,0.01030585
3,1001020200,36068,0.57782705,0.569230769,0.05263158,0.56914894
4,1001020300,36067,1.00000000,1.000000000,1.00000000,1.00000000
5,1001020400,36067,0.02552960,0.589743590,0.25000000,0.07103825


[1] "Tracts in TRACTS_TO_ZIP file: 73463"

[1] "Tracts in Florida Calculated Sinkholes (HU>0): 2976"

[1] "Florida Tracts (HU>0) joined with zips: 648985"

[1] "Unique Zips in join: 1389"

[1] "Calculated sinkholes nrows: 304910"

[1] "Merged Sinkholes File nrow: 648985"

[1] "Note: there are more zips than tracts which is why the count has expanded so rapidly.  We will deal with that below in the max risk function"


In [33]:
# For zips that didn't merge correctly we need to find the closest zip and join with that
#tracts_without_zips <- tracts_with_zips[is.na(tracts_with_zips$ZIP),]
#tracts_with_zips <- tracts_with_zips[!is.na(tracts_with_zips$ZIP),]
#tracts_without_zips
#nrow(tracts_with_zips)

# These zips are necessary as they represent all the tracts that weren't covered by the above merge but are necessary as the model was trained with that column
# although these territories aren't associated with zips we pick (via brute force Google map geocoding) the closest zip code by eyeball
tracts_with_zips[tracts_with_zips$idx==29876,]['ZIP'] <- '33071' # Everglades
tracts_with_zips[tracts_with_zips$idx==157121,]['ZIP'] <- '33033' # Islandia, FL "Ragged Keys"
tracts_with_zips[tracts_with_zips$idx==165352,]['ZIP'] <- '33033' # Islandia, FL "Eliot Key"
tracts_with_zips[tracts_with_zips$idx==185410,]['ZIP'] <- '33040' # Dry Tortuga National Park Ft Jackson Key 
tracts_with_zips[tracts_with_zips$idx==184739,]['ZIP'] <- '33040' # Dry Tortuga National Park Ft Jackson Key 
tracts_with_zips[tracts_with_zips$idx==221804,]['ZIP'] <- '33403' # Weird area near Palm Beach Gardens
tracts_with_zips[tracts_with_zips$idx==212373,]['ZIP'] <- '33478' # Loxahatchee Slough Natural Area
tracts_with_zips[tracts_with_zips$idx==257806,]['ZIP'] <- '34759' # Poinciana Park (lots of lakes)

# Create tree for searching
#tracts_with_zips_lon_lat <- as.data.frame(tracts_with_zips[,c('INTPTLON10', 'INTPTLAT10')])
#tract_zips_latlon_tree <- createTree(tracts_with_zips_lon_lat, maxDepth=50)

#tracts_within_13_miles <- function(long_lat_pt) {
#    # returns the incidents within ~13 miles of long_lat_pt.  All calculations in lat/long or lat/long approximations
#    xmin_max <- c(long_lat_pt[1] - 0.2, long_lat_pt[1] + 0.2)
#    ymin_max <- c(long_lat_pt[2] - 0.2, long_lat_pt[2] + 0.2)
#    thirteen_mile_rect <- rectLookup(tract_zips_latlon_tree, xlims=xmin_max, ylims=ymin_max)
#    return(slice(tracts_with_zips, thirteen_mile_rect))
#}

# Return zip with min geoDist between our tracts without zips and tracts_with_zips
#zip_of_min_geodist <- function(row) {
#    row_lat <- row['INTPTLAT10']
#    row_lon <- row['INTPTLON10']
#    slice_of_close_tracts <- tracts_within_13_miles(as.data.frame(c(row_lon, row_lat)))
#    geodist_row <- function(row) {
#        return(distGeo(c(row_lon, row_lat), c(row['INTPTLON10'], row['INTPTLAT10'])))
#    }
    # calculates distances to all tracts
#    tracts_with_zips$distances <- apply(slice_of_close_tracts, 1, geodist_row)
    # returns zip of tiniest distance
#    print(with(tracts_with_zips, ZIP[distances == min(distances)]))
#}
#apply(tracts_without_zips[1], 1, zip_of_min_geodist)


In [42]:
# Loading our best models created via "Sample For Risknet.r"
load(file = "predicted_sinkhole_models.bin", verbose=TRUE)

Loading objects:
  fit_RF_train_reg_125
  fit_RF_train_reg_250
  fit_RF_train_reg_500
  fit_RF_train_reg_1000


In [43]:
# We rename columns to match our prediction column names, not sure if this is necessary but doing it just in case
X_df <- tracts_with_zips[c('ZIP', 'INTPTLAT10', 'INTPTLON10', 'num_sinkholes_within_75', 'num_sinkholes_within_mile')]
prediction_columns <- c('PropertyZip', 'PropertyLatitude', 'PropertyLongitude', 
                        'num_sinkholes_within_75', 'num_sinkholes_within_mile')
names(X_df) <- prediction_columns
# our new predictions for the various thresholds
num_sinkholes_125_yhat <- predict(fit_RF_train_reg_125, X_df)
num_sinkholes_250_yhat <- predict(fit_RF_train_reg_250, X_df)
num_sinkholes_500_yhat <- predict(fit_RF_train_reg_500, X_df)
num_sinkholes_1000_yhat <- predict(fit_RF_train_reg_1000, X_df)

In [44]:
#recombine into one new big giant df
new_tracts_with_zips_predicted <- cbind(tracts_with_zips, 
                                        num_sinkholes_125_yhat, num_sinkholes_250_yhat, 
                                        num_sinkholes_500_yhat, num_sinkholes_1000_yhat)

In [45]:
sinkhole_predicted_cols <- c('num_sinkholes_125_yhat', 'num_sinkholes_250_yhat', 
                             'num_sinkholes_500_yhat', 'num_sinkholes_1000_yhat')
lookup_probability <- function(row) {
    max_probability <- 0.000016
    for(index in 1:(length(row)-1)) { # we subtract 1 for idx column
        threshold <- sinkhole_predicted_cols[index]
        num_of_sinkholes_at_threshold <- new_tracts_with_zips_predicted[row['idx'], threshold] + 1 # because indexing begins at 1 in R
        sinkholes_idx <- ifelse(num_of_sinkholes_at_threshold > 10, 10, num_of_sinkholes_at_threshold)
        prob <- sinkhole_probabilities[sinkholes_idx, index]
        max_probability <- ifelse(max_probability < prob, prob, max_probability)
    }
    return(max_probability)
}
new_tracts_with_zips_predicted$idx <- 1:nrow(new_tracts_with_zips_predicted)
probabilities <- apply(new_tracts_with_zips_predicted[c(sinkhole_predicted_cols, 'idx')], 1, lookup_probability)

In [46]:
new_tracts_with_zips_predicted_prob <- cbind(new_tracts_with_zips_predicted, probabilities)
write.table(new_tracts_with_zips_predicted_prob, file='new_tracts_with_zips_predicted_prob.csv', sep=',')

In [115]:
# remove un-needed columns from final result
cols_in_final_df <- names(new_tracts_with_zips_predicted_prob)
drops <- c('state_county_tract', 'X.x', 'idx', 'X.y', 'ZIP', 'RES_RATIO', 'BUS_RATIO', 'OTH_RATIO', 'TOT_RATIO')
names_not_in_drops <- !(cols_in_final_df %in% drops)
new_tracts_with_zips_predicted_prob <- new_tracts_with_zips_predicted_prob[,names_not_in_drops]
new_tracts_with_zips_predicted_prob$geoid <- as.character(new_tracts_with_zips_predicted_prob$geoid)

# group by max(prob) per tract (tracts can have multiple zip codes)
# Note: we're going to use all the columns except for the probabilities which we'll take the max as risk
cols_in_final_df <- names(new_tracts_with_zips_predicted_prob)
all_cols_except_prob_vec <- !(cols_in_final_df %in% c('probabilities'))
all_cols_except_prob <- cols_in_final_df[all_cols_except_prob_vec]

foo <- new_tracts_with_zips_predicted_prob[,c('STATEFP10', 'COUNTYFP10', 'TRACTCE10', 'BLOCKCE10', 
                                              'INTPTLAT10', 'INTPTLON10', 'HU10', 'geoid', 'num_sinkholes_within_mile',
                                              'num_sinkholes_within_75', 'num_sinkholes_within_5', 
                                              'num_sinkholes_within_25', 'num_sinkholes_within_125', 
                                              'num_sinkholes_1000_yhat', 'num_sinkholes_500_yhat',
                                              'num_sinkholes_250_yhat', 'num_sinkholes_125_yhat', 'probabilities')]

foo$num_sinkholes_1000_yhat <- as.character(foo$num_sinkholes_1000_yhat)
foo$num_sinkholes_500_yhat <- as.character(foo$num_sinkholes_500_yhat)
foo$num_sinkholes_250_yhat <- as.character(foo$num_sinkholes_250_yhat)
foo$num_sinkholes_125_yhat <- as.character(foo$num_sinkholes_125_yhat)

#new_tracts_with_zips_predicted_prob_dt <- data.table(new_tracts_with_zips_predicted_prob)
#new_tracts_with_zips_predicted_prob_dt[, probability=max(probabilities), by=list(all_cols_except_prob)]
agg <- aggregate(probabilities ~ ., data=foo, FUN=max)
head(agg)
nrow(agg)
#agg
nrow(state_blocks)

# to get relative probability of sinkhole occurring
agg$relative_probability <- agg$probabilities / 0.000016

# write out file for final result
write.csv(agg, file="final_output_risk_per_block.csv")

STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE10,INTPTLAT10,INTPTLON10,HU10,geoid,num_sinkholes_within_mile,num_sinkholes_within_75,num_sinkholes_within_5,num_sinkholes_within_25,num_sinkholes_within_125,num_sinkholes_1000_yhat,num_sinkholes_500_yhat,num_sinkholes_250_yhat,num_sinkholes_125_yhat,probabilities
12,77,950100,2174,30.26624,-84.74247,34,120779501002174,0,0,0,0,0,0.197911921645538,0.0880912733477405,0.0353168106342222,0.00586195900608518,1.6e-05
12,77,950100,2180,30.26727,-84.75843,1,120779501002180,0,0,0,0,0,0.197911921645538,0.0880912733477405,0.0353168106342222,0.00586195900608518,1.6e-05
12,129,10100,1028,30.27061,-84.70576,5,121290101001028,0,0,0,0,0,0.197911921645538,0.0900912733477405,0.0353168106342222,0.00586195900608518,1.6e-05
12,77,950100,2189,30.23826,-84.71542,3,120779501002189,0,0,0,0,0,0.18814116134604,0.0874691365983197,0.0353554980419085,0.00586195900608518,1.6e-05
12,77,950100,2447,30.22898,-84.72236,1,120779501002447,0,0,0,0,0,0.18814116134604,0.0874691365983197,0.0353554980419085,0.00586195900608518,1.6e-05
12,129,10100,1018,30.22969,-84.68637,41,121290101001018,0,0,0,0,0,0.18814116134604,0.0874691365983197,0.0353554980419085,0.00586195900608518,1.6e-05


[1] 589799

[1] 304910